In [78]:
import warnings
warnings.filterwarnings("ignore")

Create Dataset

In [13]:
import pandas as pd
import numpy as np

first_df=pd.read_csv('Arrests_cleaned.csv')

In [22]:
import requests, json
r = requests.get('https://data.cityofchicago.org/resource/crimes.json?$limit=600000')
data = r.json()
sec_df = pd.DataFrame(data)
sec_df = sec_df[['case_number','ward','district']]

In [66]:
merged_df = pd.merge(first_df, sec_df, on="case_number")
print(len(merged_df))
merged_df.head(1)

69054


,id,case_number,date_month,date_day,date_year,time,meridiem,race,charge_1_statute,charge_1_description,...,charge_4_statute,charge_4_description,charge_4_type,charge_4_class,CHARGES STATUTE,CHARGES DESCRIPTION,CHARGES TYPE,CHARGES CLASS,ward,district
0,30039812,JE183770,3,30,2021,12:10:00,AM,WHITE HISPANIC,720 ILCS 5.0/12-3.2-A-1,DOMESTIC BATTERY - BODILY HARM,...,NaN,NaN,NaN,NaN,720 ILCS 5.0/12-3.2-A-1 | | |,DOMESTIC BATTERY - BODILY HARM | | |,M | | |,A | | |,14,008


In [46]:
# races = {'BLACK': 1, 'WHITE': 2, 'WHITE HISPANIC': 3, 'BLACK HISPANIC': 4, 'ASIAN / PACIFIC ISLANDER': 5, 'AMER INDIAN / ALASKAN NATIVE': 6, 'UNKNOWN / REFUSED': 7}
# merged_df['race'] = merged_df['race'].apply(lambda x: races[x])
# merged_df = merged_df[['race','ward','district','date_day','date_year','charge_1_type']].dropna()
# merged_df.head(1)


,race,ward,district,date_day,date_year,charge_1_type
0,3,14,008,30,2021,M


In [68]:
races = merged_df.loc[:, "race"].unique().tolist()
race_dummies = pd.get_dummies(merged_df, columns=["race"])
races = ["race_"+r for r in races]
race_dummies = race_dummies[races]
race_dummies

,race_WHITE HISPANIC,race_BLACK,race_BLACK HISPANIC,race_WHITE,race_ASIAN / PACIFIC ISLANDER,race_AMER INDIAN / ALASKAN NATIVE,race_UNKNOWN / REFUSED
0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
69049,0,0,1,0,0,0,0
69050,0,1,0,0,0,0,0
69051,0,1,0,0,0,0,0
69052,1,0,0,0,0,0,0


In [71]:
# dummy_races = ['amer_indian/alaskan_native', \
#                'asian/pacific islander', 'black', 'black_hispanic', \
#                'unknown/refused', 'white', 'white_hispanic']
# for dummy in dummy_races:
#     merged_df[dummy] = merged_df['race']

# races = merged_df.loc[:, "race"].unique()
# race_dummies = pd.get_dummies(merged_df, columns=['race'])
# race_dummies = race_dummies[dummy_races]
# print(race_dummies)


merged_df = merged_df[['ward','district','date_day','date_year','charge_1_type']]

merged_df = pd.concat([merged_df, race_dummies], axis=1).dropna()

In [72]:
merged_df

,ward,district,date_day,date_year,charge_1_type,race_WHITE HISPANIC,race_BLACK,race_BLACK HISPANIC,race_WHITE,race_ASIAN / PACIFIC ISLANDER,race_AMER INDIAN / ALASKAN NATIVE,race_UNKNOWN / REFUSED
0,14,008,30.0,2021.0,M,1,0,0,0,0,0,0
1,22,010,23.0,2021.0,M,0,1,0,0,0,0,0
2,7,004,16.0,2021.0,F,0,1,0,0,0,0,0
3,24,011,16.0,2021.0,F,0,1,0,0,0,0,0
4,13,008,16.0,2021.0,M,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69049,22,010,7.0,2022.0,F,0,0,1,0,0,0,0
69050,34,022,7.0,2022.0,M,0,1,0,0,0,0,0
69051,9,005,7.0,2022.0,F,0,1,0,0,0,0,0
69052,7,003,7.0,2022.0,F,1,0,0,0,0,0,0


In [74]:
from sklearn.model_selection import train_test_split

x = merged_df[['race_WHITE HISPANIC','race_BLACK','race_BLACK HISPANIC','race_WHITE','race_ASIAN / PACIFIC ISLANDER','race_AMER INDIAN / ALASKAN NATIVE', \
    'race_UNKNOWN / REFUSED','ward','district','date_day','date_year']]
y = merged_df[['charge_1_type']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

KNN - Grid Search

In [80]:
from sklearn.model_selection import GridSearchCV
from  sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
params = {'n_neighbors':[i for i in range(3, 10, 2)],'weights':['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size':[i for i in range(20, 40, 5)]}
clf = GridSearchCV(knn, params, cv=5)
clf.fit(x_train,y_train)
print(clf.best_estimator_)

KNeighborsClassifier(algorithm='brute', leaf_size=20, n_neighbors=9)


In [82]:
clf.best_score_

0.568976680639178

KNN Calculations

In [88]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

scalers = [MinMaxScaler(), StandardScaler(), PCA()]
for index, scaling in enumerate(scalers):
    pipe = make_pipeline(scaling, clf.best_estimator_)
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    print(scalers[index], "score = ", accuracy_score(y_pred, y_test))

MinMaxScaler() score =  0.5657494569152788
StandardScaler() score =  0.5675597393193338
PCA() score =  0.5698769007965243


In [89]:
clf.best_estimator_.fit(x_train,y_train)
y_pred = clf.best_estimator_.predict(x_test)
print("No scaling score = ", accuracy_score(y_pred, y_test))

No scaling score =  0.5669804489500362
